In [ ]:
%%time
import malaya

In [ ]:
%%time
model = malaya.bert.load(model = 'base')

In [ ]:
%%time

strings = ['saya suka makan anjing', 'saya suka makan ayam', 'mahathir baik', '<butuh> punkog ayom hang la sakai budooooh']
x = model.vectorize(strings)

In [ ]:
model.attention(strings)

In [ ]:
model.attention(strings)

In [ ]:
x.shape

In [ ]:
import numpy as np
np.cov(x)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(x, x)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

labelled = ['saya suka makan anjing', 'saya suka makan ayam', 'mahathir baik', 'butuh la']
plt.figure(figsize = (7, 7))
g = sns.clustermap(
            cosine_similarity(x, x),
            cmap = 'Blues',
            xticklabels = labelled,
            yticklabels = labelled,
            annot = True,
        )
plt.show()

In [ ]:
labelled = ['saya suka makan anjing', 'saya suka makan ayam', 'mahathir baik', 'butuh la']
plt.figure(figsize = (7, 7))
g = sns.clustermap(
            cosine_similarity(x, x),
            cmap = 'Blues',
            xticklabels = labelled,
            yticklabels = labelled,
            annot = True,
        )
plt.show()

In [ ]:
from malaya.texts._text_functions import bert_tokenization, padding_sequence

batch_x, _, _, s_tokens = bert_tokenization(
            model._tokenizer, strings, cls = model._cls, sep = model._sep
)
attentions = model._sess.run(model.attns, feed_dict = {model.X: batch_x})

In [ ]:
s_tokens

In [ ]:
import numpy as np

combined_attentions = []
for a in attentions:
    combined_attentions.append(list(a.values())[0])
cls_attn = np.mean(combined_attentions, axis = 0).mean(axis = 2)

In [ ]:
cls_attn = np.mean(cls_attn, axis = 1)
total_weights = np.sum(cls_attn, axis = -1, keepdims = True)
attn = cls_attn / total_weights

In [ ]:
attn.shape

In [ ]:
maxlen = max([len(s) for s in s_tokens])
s_tokens = padding_sequence(s_tokens, maxlen, pad_int = model._sep)

In [ ]:
len(s_tokens[-1])

In [ ]:
paired_tokens = list(zip(s_tokens[-1], attn[-1]))
paired_tokens

In [ ]:
def merge_wordpiece_tokens(paired_tokens):
    new_paired_tokens = []
    n_tokens = len(paired_tokens)
    rejected = ['<cls>', '<sep>']

    i = 0

    while i < n_tokens:
        
        current_token, current_weight = paired_tokens[i]
        if not current_token.startswith('▁') and current_token not in rejected:
            previous_token, previous_weight = new_paired_tokens.pop()
            merged_token = previous_token
            merged_weight = [previous_weight]
            while not current_token.startswith('▁') and current_token not in rejected:
                merged_token = merged_token + current_token.replace('▁', '')
                merged_weight.append(current_weight)
                i = i + 1
                current_token, current_weight = paired_tokens[i]
            merged_weight = np.mean(merged_weight)
            new_paired_tokens.append((merged_token, merged_weight))

        else:
            new_paired_tokens.append((current_token, current_weight))
            i = i + 1

    words = [
        i[0].replace('▁','')
        for i in new_paired_tokens
        if i[0] not in ['<cls>', '<sep>', '<pad>']
    ]
    weights = [
        i[1]
        for i in new_paired_tokens
        if i[0] not in ['<cls>', '<sep>', '<pad>']
    ]
    weights = np.array(weights)
    weights = weights / np.sum(weights)
    return list(zip(words, weights))

In [ ]:
merge_wordpiece_tokens(paired_tokens)